#### Tool Error Handling

In [1]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage
from dotenv import load_dotenv
import os
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.agents.structured_output import ToolStrategy

C:\Users\Ankit\AppData\Roaming\Python\Python314\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
# Load .env variables
load_dotenv()

# Read API key
gpt_api_key = os.getenv("OPENAI_API_KEY")

# Set the environment variable that OpenAI expects
if gpt_api_key:
    print("API Loaded Succesfully!")
else:
    raise ValueError("GPT_API not found in .env file")

API Loaded Succesfully!


In [9]:
from langchain.tools import tool

@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    if True:
        raise RuntimeError("Forced failure for testing!")
    return f"Weather in {location}: Sunny, 72°F"

In [10]:
@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

In [11]:
agent = create_agent(
    model="gpt-4o-mini",
    tools=[search, get_weather],
    middleware=[handle_tool_errors]
)

In [12]:
question = "what is the weather in san andreas"

response = agent.invoke(
    {"messages": [{"role": "user", "content": question}]}
)

In [17]:
response["messages"][2].content

'Tool error: Please check your input and try again. (Forced failure for testing!)'

In [18]:
response["messages"][-1].content

"It seems I'm unable to retrieve the weather information directly. However, you can check a reliable weather website or app for the current weather in San Andreas, California. If you have any other questions or need further assistance, feel free to ask!"